In [1]:
from dataclasses import dataclass
from enum import StrEnum
from datetime import datetime, timedelta, date
import polars as pl
from collections import defaultdict, Counter
from typing import Tuple
from common.constants.column_types import (
    CPZP_SCHEMA,
    OZP_SCHEMA,
    POHLAVI_CPZP,
    TYP_UDALOSTI,
)
from common.constants.column_names import SHARED_COLUMNS, OZP_COLUMNS, CPZP_COLUMNS
import pickle
from common.constants.objects import (
    Person,
    Gender,
    AgeCohort,
    Prescription,
    PrescriptionType,
)
import matplotlib.pyplot as plt
import numpy as np
import os
from common.utils import (
    draw_chart,
    filter_by_date_range,
)

pl.Config.set_tbl_rows(20)
pl.Config.set_tbl_cols(60)

from typing import Any
import matplotlib.dates as mdates
from matplotlib.patches import Patch
from scipy.stats import fisher_exact

pl.Config.set_tbl_rows(-1)
import pandas as pd

In [2]:
# CONSTANTS
POJISTOVNA = "cpzp"

In [3]:
with open(f"./DATACON_data/{POJISTOVNA}_persons2.pkl", "rb") as f:
    persons: list[Person] = pickle.load(f)

In [ ]:
print(
    41975
    + 1435
    + 59069
    + 56177
    + 78693
    + 96351
    + 36559
    + 97513
    + 3397
    + 10802
    + 1281
    + 2876
)

486128


In [6]:
VAX_PERIOD_IN_DAYS = 30
START_DATE = date(2015, 1, 1)
END_DATE = date(2025, 1, 1)
INJECTION_FORMS = {"Injekční suspenze", "Injekční/infuzní roztok"}
PERIOD = 364

# --- Maps --------------------------------------------------------------------
vax_dates_map: dict[AgeCohort, dict[int, list[date]]] = defaultdict(
    lambda: defaultdict(list)
)
max_vax_intensity_map: dict[AgeCohort, dict[int, date]] = defaultdict(dict)

novax_ppl_predpisy_map: dict[AgeCohort, dict[date, int]] = defaultdict(
    lambda: defaultdict(int)
)
novax_ppl_prvopredpisy_map: dict[AgeCohort, dict[date, int]] = defaultdict(
    lambda: defaultdict(int)
)
novax_ppl_prednison_equivs_map: dict[AgeCohort, dict[date, float]] = defaultdict(
    lambda: defaultdict(float)
)
novax_ppl_imunosupresivy_map: dict[AgeCohort, dict[date, int]] = defaultdict(
    lambda: defaultdict(int)
)

vax_ppl_prvopredpisy_map = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))
vax_ppl_predpisy_map = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))
vax_ppl_prednison_equivs_map = defaultdict(
    lambda: defaultdict(lambda: defaultdict(int))
)
vax_ppl_imunosupresivy_map = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))


# --- Helpers -----------------------------------------------------------------
def is_injection(form: str) -> bool:
    return form in INJECTION_FORMS


def skip_person_for_novax(p) -> bool:
    return (
        (p.zahajeni_pojisteni > START_DATE)
        or (p.ukonceni_pojisteni < END_DATE)
        or p.died_at
        or bool(p.vaccines)
        or not p.prescriptions
    )


def skip_person_for_vax(p) -> bool:
    return (
        (p.zahajeni_pojisteni > START_DATE)
        or (p.ukonceni_pojisteni < END_DATE)
        or p.died_at
        or (not p.vaccines)
        or (not p.prescriptions)
    )


def collapse_injections(prescriptions):
    """Yield prescriptions but ignore additional injections within 14 days."""
    last_inj_date = date.min
    for pr in prescriptions:
        if is_injection(pr.lekova_forma):
            if abs((last_inj_date - pr.date).days) < 14:
                continue
            last_inj_date = pr.date
        yield pr


def safe_div(a: float, b: float) -> float:
    return np.nan if b == 0 else a / b


def pvalue_from_df(df: pl.DataFrame) -> float:
    """Fisher exact test on Polars df with rows očkovaní/neočkovaní, cols před/po"""
    table = [
        [
            int(df.filter(pl.col("group") == "očkovaní")["před"][0]),
            int(df.filter(pl.col("group") == "očkovaní")["po"][0]),
        ],
        [
            int(df.filter(pl.col("group") == "neočkovaní")["před"][0]),
            int(df.filter(pl.col("group") == "neočkovaní")["po"][0]),
        ],
    ]
    return fisher_exact(table, alternative="less")[1]


def before_after_df(
    vax_map: dict[int, float], novax_map: dict[int, float], pivot: int = 0
) -> pl.DataFrame:
    def sums(m):
        before = sum(v for d, v in m.items() if d < pivot)
        after = sum(v for d, v in m.items() if d >= pivot)
        return before, after

    vb, va = sums(vax_map)
    nb, na = sums(novax_map)
    return pl.DataFrame(
        {"group": ["očkovaní", "neočkovaní"], "před": [vb, nb], "po": [va, na]}
    )


def before_after_sums(
    dates_map: dict[int, float | int], pivot_day: int = 0
) -> dict[str, float]:
    before = sum(v for d, v in dates_map.items() if d < pivot_day)
    after = sum(v for d, v in dates_map.items() if d >= pivot_day)
    return {"před": before, "po": after}


# --- Max vaccination dates by cohort ----------------------------------------
for person in persons:
    if person.died_at or not person.vaccines:
        continue
    for v in person.vaccines:
        vax_dates_map[v.age_cohort][v.dose_number].append(v.date)

for cohort, doses in vax_dates_map.items():
    for dose, dates in doses.items():
        if dates:
            max_vax_intensity_map[cohort][dose] = Counter(dates).most_common(1)[0][0]

# --- NOVAX metrics -----------------------------------------------------------


for p in persons:
    if skip_person_for_novax(p):
        continue

    for pr in collapse_injections(p.prescriptions):
        cohort = pr.age_cohort_at_prescription
        if pr.prescription_type == PrescriptionType.IMUNOSUPRESSIVE:
            novax_ppl_imunosupresivy_map[cohort][pr.date] += 1

        novax_ppl_predpisy_map[cohort][pr.date] += 1
        novax_ppl_prednison_equivs_map[cohort][pr.date] += pr.prednison_equiv

    first = p.prescriptions[0]
    novax_ppl_prvopredpisy_map[first.age_cohort_at_prescription][first.date] += 1


# --- VAX metrics -------------------------------------------------------------
for p in persons:
    if skip_person_for_vax(p):
        continue

    for v in p.vaccines:
        max_int_date = max_vax_intensity_map[v.age_cohort][v.dose_number]
        if abs((v.date - max_int_date).days) > VAX_PERIOD_IN_DAYS:
            continue

        # prescriptions relative to this vax
        for pr in collapse_injections(p.prescriptions):
            rel_day = (pr.date - v.date).days
            if pr.prescription_type == PrescriptionType.IMUNOSUPRESSIVE:
                vax_ppl_imunosupresivy_map[v.age_cohort][v.dose_number][rel_day] += 1

            vax_ppl_predpisy_map[v.age_cohort][v.dose_number][rel_day] += 1
            vax_ppl_prednison_equivs_map[v.age_cohort][v.dose_number][
                rel_day
            ] += pr.prednison_equiv

        first = min(p.prescriptions, key=lambda x: x.date)
        rel_first = (first.date - v.date).days
        vax_ppl_prvopredpisy_map[v.age_cohort][v.dose_number][rel_first] += 1

In [7]:
novax_count = 0


celkem = 0
for p in persons:
    if p.died_at:
        continue
    if p.zahajeni_pojisteni > START_DATE:
        continue
    if p.ukonceni_pojisteni < END_DATE:
        continue
    if p.age_cohort == AgeCohort.BETWEEN_12_AND_30:
        celkem += 1

for p in persons:
    if (
        (p.zahajeni_pojisteni > START_DATE)
        or (p.ukonceni_pojisteni < END_DATE)
        or p.died_at
        or (p.vaccines)
    ):
        continue
    if p.age_cohort == AgeCohort.BETWEEN_12_AND_30:
        novax_count += 1

vax_count = 0
for p in persons:
    if (
        (p.zahajeni_pojisteni > START_DATE)
        or (p.ukonceni_pojisteni < END_DATE)
        or p.died_at
        or (not p.vaccines)
    ):
        continue
    if p.age_cohort == AgeCohort.BETWEEN_12_AND_30:
        vax_count += 1


print(novax_count)
print(vax_count)
print(vax_count + novax_count)
print(celkem)

94040
89932
183972
183972


In [19]:
number_of_ppl_novax = [
    1
    for p in persons
    if not skip_person_for_novax(p) and p.age_cohort == AgeCohort.BETWEEN_30_AND_50
]
print(len(number_of_ppl_novax))

31058


In [10]:
# --- Aggregation: before/after windows --------------------------------------
rows = []
dose = 1

for cohort in AgeCohort:
    print(
        f"COHORT: {cohort}, DOSE: {dose}, MAX_VAX_INTENSITY: {max_vax_intensity_map[cohort][dose]}"
    )
    rozhodne = max_vax_intensity_map[cohort][dose]

    start, end = rozhodne - timedelta(days=PERIOD), rozhodne + timedelta(days=PERIOD)
    rel_days = [(start + timedelta(days=i)) for i in range((end - start).days + 1)]

    def novax_window(src_map):
        return {(d - rozhodne).days: src_map[cohort].get(d, 0) for d in rel_days}

    def vax_window(src_map):
        return {
            day: src_map[cohort][dose].get(day, 0) for day in range(-PERIOD, PERIOD)
        }

    metrics = {
        "predpisy": (
            vax_window(vax_ppl_predpisy_map),
            novax_window(novax_ppl_predpisy_map),
        ),
        "prvopredpisy": (
            vax_window(vax_ppl_prvopredpisy_map),
            novax_window(novax_ppl_prvopredpisy_map),
        ),
        "kortikoidy": (
            vax_window(vax_ppl_prednison_equivs_map),
            novax_window(novax_ppl_prednison_equivs_map),
        ),
        "imunosupresivy": (
            vax_window(vax_ppl_imunosupresivy_map),
            novax_window(novax_ppl_imunosupresivy_map),
        ),
    }

    for metric_name, (vax_map, novax_map) in metrics.items():
        df = before_after_df(vax_map, novax_map)

        vax_b, vax_a = (
            df.filter(pl.col("group") == "očkovaní").select(["před", "po"]).row(0)
        )
        nov_b, nov_a = (
            df.filter(pl.col("group") == "neočkovaní").select(["před", "po"]).row(0)
        )

        vax_inc = safe_div(vax_a, vax_b) * 100 if vax_b else np.nan
        novax_inc = safe_div(nov_a, nov_b) * 100 if nov_b else np.nan
        diff = (
            (vax_inc - novax_inc)
            if (np.isfinite(vax_inc) and np.isfinite(novax_inc))
            else np.nan
        )
        p_val = pvalue_from_df(df)

        ratio = np.nan
        if vax_b and nov_b and np.isfinite(vax_inc) and np.isfinite(novax_inc):
            ratio = safe_div(vax_a / vax_b, nov_a / nov_b)

        rows.append(
            {
                "period_days": PERIOD,
                "age_cohort": str(cohort),
                "vax_dose": int(dose),
                "metric": metric_name,
                "vax_increase": vax_inc,
                "novax_increase": novax_inc,
                "diff": diff,
                "p_value": p_val,
                "vax_before": vax_b,
                "vax_after": vax_a,
                "novax_before": nov_b,
                "novax_after": nov_a,
                "vax_vs_novax_ratio": ratio,
            }
        )

huge_df = (
    pl.DataFrame(rows)
    .sort(["age_cohort"])
    .filter((pl.col("vax_dose") == 1) & (pl.col("metric") == "prvopredpisy"))
)


huge_df

COHORT: less_than_12, DOSE: 1, MAX_VAX_INTENSITY: 2021-12-21
COHORT: between_12_and_30, DOSE: 1, MAX_VAX_INTENSITY: 2021-06-09
COHORT: between_30_and_50, DOSE: 1, MAX_VAX_INTENSITY: 2021-05-19
COHORT: between_50_and_70, DOSE: 1, MAX_VAX_INTENSITY: 2021-05-10
COHORT: more_than_70, DOSE: 1, MAX_VAX_INTENSITY: 2021-03-11


period_days,age_cohort,vax_dose,metric,vax_increase,novax_increase,diff,p_value,vax_before,vax_after,novax_before,novax_after,vax_vs_novax_ratio
i64,str,i64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64
364,"""between_12_and_30""",1,"""prvopredpisy""",117.295189,120.907298,-3.612109,0.691446,769.0,902.0,1014.0,1226.0,0.970125
364,"""between_30_and_50""",1,"""prvopredpisy""",94.86105,104.637847,-9.776797,0.999027,4534.0,4301.0,3493.0,3655.0,0.906565
364,"""between_50_and_70""",1,"""prvopredpisy""",91.512423,103.360935,-11.848512,0.999645,5997.0,5488.0,2053.0,2122.0,0.885368
364,"""less_than_12""",1,"""prvopredpisy""",87.323944,124.276169,-36.952226,0.980282,71.0,62.0,898.0,1116.0,0.70266
364,"""more_than_70""",1,"""prvopredpisy""",105.074821,129.337539,-24.262718,0.994684,1537.0,1615.0,317.0,410.0,0.812408
